In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import pytesseract
from PIL import Image, ImageDraw

pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract'

def mask_text_cv2(cv2_image):
    # Convert the cv2 image (BGR) to PIL Image (RGB)
    rgb_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)

    # Use pytesseract to do OCR on the image
    text_data = pytesseract.image_to_data(pil_image)

    # Create a drawing context
    draw = ImageDraw.Draw(pil_image)
    print(text_data.split('\n')[0])

    # Process the OCR data
    for line in text_data.split('\n')[1:]:
        if line.strip() == '':
            continue

        parts = line.split()
        print(parts)
        if len(parts) >= 12:
            x, y, width, height = map(int, parts[6:10])
            # Draw a white rectangle over the detected text
            draw.rectangle([x, y, x + width, y + height], fill="white")

    # Convert PIL Image back to cv2 format (BGR)
    masked_cv2_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    return masked_cv2_image


def check_match_images(src_img, web_img, visualize=False):
    # Read the images
    image_b = cv2.imread(web_img)
    image_b = mask_text_cv2(image_b)
    image_a = cv2.imread(src_img)

    # SIFT detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors
    keypoints_a, descriptors_a = sift.detectAndCompute(image_a, None)
    keypoints_b, descriptors_b = sift.detectAndCompute(image_b, None)

    # FLANN based matcher
    index_params = dict(algorithm=1, trees=5)
    search_params = dict()
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(descriptors_a, descriptors_b, k=2)

    # Keep good matches: Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10: # adjust this threshold

        image_matches = cv2.drawMatches(image_a, keypoints_a, image_b, keypoints_b, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
        src_pts = np.float32([keypoints_a[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_b[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Use the homography matrix M to transform the corners of Image A to Image B's plane
        h, w = image_a.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)

        # Draw the transformed image on Image B
        image_b_with_a = cv2.polylines(image_b, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

        # gray = cv2.cvtColor(image_b_with_a, cv2.COLOR_BGR2GRAY)
        if visualize:
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.axis('off')
            plt.imshow(image_matches)
            plt.show()

        hb, wb = image_b.shape[:2]
        print(hb, wb)
        print(dst)

        # Extract scale and translation (approximate)
        scale_x = np.linalg.norm(dst[1] - dst[0]) / hb
        scale_y = np.linalg.norm(dst[2] - dst[1]) / wb
        translation = dst[0][0] / np.array([hb, wb])

        print(f"Relative height: {scale_x}, Relative width: {scale_y}")
        print(f"Top-Left Corner Coordinate: {translation}")
        return scale_x, scale_y, translation.tolist()
    else:
        print("Image not found!")
        return None, None, [None, None]
    

# check_match_images('../trial_dataset/rick.jpg', './diyi.png')
check_match_images('../trial_dataset/rick.jpg', './diyi_gpt4.png', True)

In [40]:
from PIL import Image
import numpy as np

file_path = "./diyi_gpt4.png"
template = file_path[:-4] + "{color}" + file_path[-4:]

image = Image.open(template.format(color="")).convert("RGB")
image_array = np.array(image)

image_red = Image.open(template.format(color="_red")).convert("RGB")
image_array_red = np.array(image_red)

image_blue = Image.open(template.format(color="_blue")).convert("RGB")
image_array_blue = np.array(image_blue)

./diyi_gpt4{color}.png


In [60]:
is_image = (image_array_red[:, :, 0] >= 250) & (image_array_red[:, :, 1] <= 5) & (image_array_red[:, :, 2] <= 5) & (image_array_blue[:, :, 0] <= 5) & (image_array_blue[:, :, 1] <= 5) & (image_array_blue[:, :, 2] >= 250)
is_image_coordinates = np.column_stack(np.where(is_image))

In [62]:
print(np.min(is_image_coordinates[:, 0]), np.max(is_image_coordinates[:, 0]), np.min(is_image_coordinates[:, 1]), np.max(is_image_coordinates[:, 1]))

20 119 1160 1259


In [5]:
import cv2
import os
from paddleocr import PaddleOCR, draw_ocr
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

ocr = PaddleOCR(use_angle_cls=True, lang="en", show_log = False)


def get_ocr_blocks(image_path):
    # This function will use OCR to extract text blocks and their bounding boxes from an image
    image = cv2.imread(image_path)
    img_h, img_w, _ = image.shape
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    blocks = []
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 50:  # Consider blocks with confidence > 60%
            (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
            text = data['text'][i].strip()
            blocks.append({'text': text, 'bbox': (x / img_w, y / img_h, w / img_w, h / img_h)})
    return blocks

def is_duplicate(new_block, existing_blocks, threshold=0.1):
    new_x, new_y, new_w, new_h = new_block['bbox']
    for block in existing_blocks:
        existing_x, existing_y, existing_w, existing_h = block['bbox']
        
        # Check if the new block is close to an existing block in terms of position and size
        if (abs(new_x - existing_x) < threshold and abs(new_y - existing_y) < threshold and
            abs(new_w - existing_w) < threshold and abs(new_h - existing_h) < threshold):
            return True
    return False

def get_ppocr_blocks(image_path, segment_height=1280, overlap=128):
    # Read the image
    image = cv2.imread(image_path)
    img_h, img_w, _ = image.shape
    
    # Initialize an empty list to store all OCR results
    all_blocks = []

    current_start_y = 0
    while current_start_y + overlap < img_h:
        # Calculate segment boundaries
        start_y = current_start_y
        end_y = min(current_start_y + segment_height, img_h)

        # Crop the image
        cropped_image = image[start_y:end_y, :, :]

        # Save the cropped image temporarily
        temp_path = f"temp_cropped_{start_y}.jpg"
        cv2.imwrite(temp_path, cropped_image)

        # Perform OCR on the cropped image
        result = ocr.ocr(temp_path, cls=True)[0]
        if result is None:
            current_start_y = end_y - overlap
            os.remove(temp_path)
            continue

        # Delete the temporary file
        os.remove(temp_path)

        # Process OCR results and adjust coordinates
        for i in range(len(result)):
            x, y, w, h = result[i][0][0][0], result[i][0][0][1], result[i][0][2][0] - result[i][0][0][0], result[i][0][2][1] - result[i][0][0][1]
            text = result[i][1][0].strip()
            adjusted_bbox = (x / img_w, (y + start_y) / img_h, w / img_w, h / img_h)
            new_block = {'text': text, 'bbox': adjusted_bbox}
            
            # Check if the new block is a duplicate
            if not is_duplicate(new_block, all_blocks):
                all_blocks.append(new_block)

        # Update the start_y for the next crop
        current_start_y = end_y - overlap

        print(current_start_y)
        print(all_blocks)

    return all_blocks

def match_blocks(blocks1, blocks2, v_scale=0.1):
    # This function will match blocks between two sets based on text similarity, spatial location, and size similarity
    matched_blocks = []
    max_distance = (1 + v_scale**2)**0.5
    
    for block1 in blocks1:
        best_match = None
        highest_score = 0
        
        for block2 in blocks2:
            # Text similarity
            text_similarity = SequenceMatcher(None, block1['text'], block2['text']).ratio()
            
            if text_similarity > 0.8:  # Text must be similar above a threshold
                
                # Spatial proximity (normalized by image dimensions for example)
                spatial_proximity = 1 - ((block1['bbox'][0] - block2['bbox'][0])**2 + (block1['bbox'][1] * v_scale - block2['bbox'][1] * v_scale)**2)**0.5 / max_distance
                
                # Size similarity
                # size_similarity = 1 - abs(block1['bbox'][2]*block1['bbox'][3] - block2['bbox'][2]*block2['bbox'][3]) / max(block1['bbox'][2]*block1['bbox'][3], block2['bbox'][2]*block2['bbox'][3])

                # Combine the scores with weights as needed
                # combined_score = (text_similarity * 0.6) + (spatial_proximity * 0.2) + (size_similarity * 0.2)
                combined_score = (text_similarity * 0.6) + (spatial_proximity * 0.4)

                print(block2)
                print(combined_score)

                if combined_score > highest_score:
                    highest_score = combined_score
                    best_match = block2

        if best_match:
            matched_blocks.append((block1, best_match))
        
        break
    
    return matched_blocks


def calculate_positional_score(bbox1, bbox2, v_scale=0.1):
    max_distance = (1 + v_scale**2)**0.5

    # Calculate the Euclidean distance between the center points of two bounding boxes
    center1 = (bbox1[0] + bbox1[2] / 2, bbox1[1] + bbox1[3] / 2)
    center2 = (bbox2[0] + bbox2[2] / 2, bbox2[1] + bbox2[3] / 2)
    distance = ((center1[0] - center2[0]) ** 2 + (center1[1] * v_scale - center2[1] * v_scale) ** 2) ** 0.5
    
    # Normalize distance based on a predefined max distance, this value could be tuned
    normalized_distance = min(distance / max_distance, 1)
    
    # Calculate score using exponential decay
    score = 1 - normalized_distance
    
    return score

In [6]:
blocks1 = get_ppocr_blocks('./diyi_gpt4.png')
blocks2 = get_ppocr_blocks('./diyi.png')

592
[{'text': 'Diyi Yang', 'bbox': (0.0140625, 0.0763888888888889, 0.08671875, 0.043055555555555555)}, {'text': 'diyiy@stanford.edu', 'bbox': (0.01484375, 0.18888888888888888, 0.109375, 0.02361111111111111)}, {'text': 'Stanford University', 'bbox': (0.01328125, 0.33055555555555555, 0.1078125, 0.027777777777777776)}, {'text': 'Home', 'bbox': (0.01328125, 0.43333333333333335, 0.03046875, 0.020833333333333332)}, {'text': 'Teaching', 'bbox': (0.16796875, 0.43194444444444446, 0.0421875, 0.025)}, {'text': 'Rehearsal: Simulating Chat to Facilitate Conflict Resolution', 'bbox': (0.01484375, 0.5458333333333333, 0.2453125, 0.018055555555555554)}, {'text': 'Using Large Language Models in Psychology', 'bbox': (0.503125, 0.5680555555555555, 0.19140625, 0.027777777777777776)}, {'text': 'Publications', 'bbox': (0.0140625, 0.6125, 0.0859375, 0.029166666666666667)}]
1152
[{'text': 'Diyi Yang', 'bbox': (0.61171875, 0.006784771956275914, 0.1046875, 0.007161703731624576)}, {'text': 'Home', 'bbox': (0.2164

In [7]:
def group_blocks_by_row(blocks, line_overlap_threshold=0.5):
    """
    Group blocks into rows based on their bounding box y-coordinates.
    Blocks that have y-overlapping bounding boxes within a threshold are considered to be on the same row.

    :param blocks: List of block dictionaries with 'bbox' as one of the keys.
    :param line_overlap_threshold: Threshold for considering blocks to be on the same line (relative to image height).
    :return: A list of lists of blocks, with each inner list representing a row.
    """
    # Sort blocks by the top y-coordinate
    sorted_blocks = sorted(blocks, key=lambda b: b['bbox'][1])
    
    rows = []
    current_row = []
    
    for block in sorted_blocks:
        # If current_row is empty, start a new row with the current block
        if not current_row:
            current_row.append(block)
        else:
            # Compare the current block with the last block in the current row
            last_block_in_row = current_row[-1]
            # Calculate the vertical overlap between the two blocks
            top_y_current = block['bbox'][1]
            bottom_y_last = last_block_in_row['bbox'][1] + last_block_in_row['bbox'][3]
            vertical_overlap = max(0, bottom_y_last - top_y_current)
            
            # If there is enough overlap, add the block to the current row
            if vertical_overlap > line_overlap_threshold * min(last_block_in_row['bbox'][3], block['bbox'][3]):
                current_row.append(block)
            else:
                # Otherwise, the current block starts a new row
                rows.extend(current_row)
                current_row = [block]
    
    # Add the last row if it's not empty
    if current_row:
        rows.extend(current_row)
    
    return rows


grouped_blocks1 = group_blocks_by_row(blocks1)
grouped_blocks2 = group_blocks_by_row(blocks2)

In [8]:
print(len(grouped_blocks1), len(grouped_blocks2))

8 36


In [10]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def calculate_similarity(block1, block2, max_distance=1.42):
    text_similarity = SequenceMatcher(None, block1['text'], block2['text']).ratio()
    spatial_proximity = 1 - ((block1['bbox'][0] - block2['bbox'][0])**2 + (block1['bbox'][1] - block2['bbox'][1])**2)**0.5 / max_distance
    combined_score = (text_similarity * 1.0) + (spatial_proximity * 0.0)
    return combined_score

def adjust_cost_for_context(cost_matrix, consecutive_bonus=1.0, window_size=20):
    n, m = cost_matrix.shape
    adjusted_cost_matrix = np.copy(cost_matrix)

    for i in range(n):
        for j in range(m):
            bonus = 0
            if adjusted_cost_matrix[i][j] >= -0.5:
                continue
            # Check left context
            for k in range(1, window_size + 1):
                if i >= k and j >= k:
                    # bonus += consecutive_bonus * (cost_matrix[i-k, j-k] < 0)
                    bonus += consecutive_bonus * cost_matrix[i-k, j-k]
            # Check right context
            for k in range(1, window_size + 1):
                if i + k < n and j + k < m:
                    # bonus += consecutive_bonus * (cost_matrix[i+k, j+k] < 0
                    bonus += consecutive_bonus * cost_matrix[i+k, j+k]
            adjusted_cost_matrix[i][j] += bonus
    return adjusted_cost_matrix

def create_cost_matrix(A, B):
    n = len(A)
    m = len(B)
    cost_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            cost_matrix[i, j] = -calculate_similarity(A[i], B[j])
    return cost_matrix

def find_maximum_matching(A, B, consecutive_bonus, window_size):
    cost_matrix = create_cost_matrix(A, B)
    cost_matrix = adjust_cost_for_context(cost_matrix, consecutive_bonus, window_size)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return list(zip(row_ind, col_ind))

# Example usage
A = grouped_blocks1
B = grouped_blocks2

matching = find_maximum_matching(A, B, 0.1, 1)
matched_list = []
print("Matching pairs:")
for i, j in matching:
    print(f"{A[i]} matched with {B[j]}")
    matched_list.append([A[i]['bbox'], B[j]['bbox']])

Matching pairs:
{'text': 'Diyi Yang', 'bbox': (0.0140625, 0.0763888888888889, 0.08671875, 0.043055555555555555)} matched with {'text': 'Diyi Yang', 'bbox': (0.61171875, 0.006784771956275914, 0.1046875, 0.007161703731624576)}
{'text': 'diyiy@stanford.edu', 'bbox': (0.01484375, 0.18888888888888888, 0.109375, 0.02361111111111111)} matched with {'text': 'Diyi Yang,Robert Kraut and John Levine', 'bbox': (0.178125, 0.9053901243874859, 0.225, 0.00395778364116095)}
{'text': 'Stanford University', 'bbox': (0.01328125, 0.33055555555555555, 0.1078125, 0.027777777777777776)} matched with {'text': 'Best Paper Honorable Mention', 'bbox': (0.17890625, 0.6093102148511119, 0.175, 0.003392385978137957)}
{'text': 'Teaching', 'bbox': (0.16796875, 0.43194444444444446, 0.0421875, 0.025)} matched with {'text': 'Teaching', 'bbox': (0.73515625, 0.04711647191858274, 0.05546875, 0.0054655107425555976)}
{'text': 'Home', 'bbox': (0.01328125, 0.43333333333333335, 0.03046875, 0.020833333333333332)} matched with {'te

In [13]:
import cv2
import numpy as np
import random

def draw_matched_bboxes(img1, img2, matched_bboxes):
    # Create copies of images to draw on
    img1_drawn = img1.copy()
    img2_drawn = img2.copy()

    h1, w1, _ = img1.shape
    h2, w2, _ = img2.shape
    

    # Iterate over matched bounding boxes
    for bbox_pair in matched_bboxes:
        # Random color for each pair
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Ensure that the bounding box coordinates are integers
        bbox1 = [int(bbox_pair[0][0] * w1), int(bbox_pair[0][1] * h1), int(bbox_pair[0][2] * w1), int(bbox_pair[0][3] * h1)]
        bbox2 = [int(bbox_pair[1][0] * w2), int(bbox_pair[1][1] * h2), int(bbox_pair[1][2] * w2), int(bbox_pair[1][3] * h2)]

        # Draw bbox on the first image
        top_left_1 = (bbox1[0], bbox1[1])
        bottom_right_1 = (bbox1[0] + bbox1[2], bbox1[1] + bbox1[3])
        img1_drawn = cv2.rectangle(img1_drawn, top_left_1, bottom_right_1, color, 2)

        # Draw bbox on the second image
        top_left_2 = (bbox2[0], bbox2[1])
        bottom_right_2 = (bbox2[0] + bbox2[2], bbox2[1] + bbox2[3])
        img2_drawn = cv2.rectangle(img2_drawn, top_left_2, bottom_right_2, color, 2)

    return img1_drawn, img2_drawn

img1 = cv2.imread('./diyi_gpt4.png')
img2 = cv2.imread('./diyi.png')

img1_with_boxes, img2_with_boxes = draw_matched_bboxes(img1, img2, matched_list)

cv2.imwrite('./diyi_gpt4_demo.png', img1_with_boxes)
cv2.imwrite('./diyi_demo.png', img2_with_boxes)

True

In [15]:
import cv2
from paddleocr import PaddleOCR
import os



In [16]:
import cv2

get_ppocr_blocks('./diyi_gpt4.jpg')

592
[{'text': 'Diyi Yang', 'bbox': (0.01484375, 0.07361111111111111, 0.0984375, 0.044444444444444446)}, {'text': 'diyiy@stanford.edu', 'bbox': (0.015625, 0.18888888888888888, 0.12265625, 0.02638888888888889)}, {'text': 'Natural Language Processing Group', 'bbox': (0.01328125, 0.2847222222222222, 0.22734375, 0.027777777777777776)}, {'text': 'Stanford University', 'bbox': (0.01484375, 0.3347222222222222, 0.11953125, 0.02361111111111111)}, {'text': 'Publications', 'bbox': (0.10859375, 0.4361111111111111, 0.06328125, 0.019444444444444445)}, {'text': 'Rehearsal: Simulating Chat to Facilitate Conflict Resolution', 'bbox': (0.01484375, 0.5444444444444444, 0.27578125, 0.02361111111111111)}, {'text': 'Using Large Language Models in Psychology', 'bbox': (0.50546875, 0.5736111111111111, 0.209375, 0.02361111111111111)}, {'text': 'Publications', 'bbox': (0.015625, 0.6194444444444445, 0.09765625, 0.02638888888888889)}]


[{'text': 'Diyi Yang',
  'bbox': (0.01484375, 0.07361111111111111, 0.0984375, 0.044444444444444446)},
 {'text': 'diyiy@stanford.edu',
  'bbox': (0.015625, 0.18888888888888888, 0.12265625, 0.02638888888888889)},
 {'text': 'Natural Language Processing Group',
  'bbox': (0.01328125, 0.2847222222222222, 0.22734375, 0.027777777777777776)},
 {'text': 'Stanford University',
  'bbox': (0.01484375, 0.3347222222222222, 0.11953125, 0.02361111111111111)},
 {'text': 'Publications',
  'bbox': (0.10859375, 0.4361111111111111, 0.06328125, 0.019444444444444445)},
 {'text': 'Rehearsal: Simulating Chat to Facilitate Conflict Resolution',
  'bbox': (0.01484375, 0.5444444444444444, 0.27578125, 0.02361111111111111)},
 {'text': 'Using Large Language Models in Psychology',
  'bbox': (0.50546875, 0.5736111111111111, 0.209375, 0.02361111111111111)},
 {'text': 'Publications',
  'bbox': (0.015625, 0.6194444444444445, 0.09765625, 0.02638888888888889)}]

In [ ]:
import easyocr
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext('./diyi_gpt4.jpg')

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
